# SciTeX Gen Module - Core Generation Utilities

This comprehensive notebook demonstrates the SciTeX gen module capabilities, covering core generation utilities and helper functions.

## Features Covered

### Core Utilities
* Data normalization and transformation
* Array dimension handling
* Type checking and validation
* Shell command execution

### Development Tools
* Configuration printing
* Module inspection
* Environment checking
* Caching mechanisms

### File Operations
* Symlink management
* Text processing
* XML/JSON conversion
* Path utilities

In [ ]:
# Detect notebook name for output directory
import os
from pathlib import Path

# Get notebook name (for papermill compatibility)
notebook_name = "02_scitex_gen"
if 'PAPERMILL_NOTEBOOK_NAME' in os.environ:
    notebook_name = Path(os.environ['PAPERMILL_NOTEBOOK_NAME']).stem


In [ ]:
# Memory management for automated execution
import gc
import matplotlib
matplotlib.use('Agg')  # Non-interactive backend
import matplotlib.pyplot as plt
plt.ioff()  # Turn off interactive mode

# Function to clean up matplotlib
def cleanup_plt():
    plt.close('all')
    gc.collect()


In [ ]:
import sys
sys.path.insert(0, '../src')
import scitex
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import tempfile
import os

# Set up example data directory
data_dir = Path("./gen_examples")
data_dir.mkdir(exist_ok=True)


In [ ]:
# Path compatibility helper
import os
from pathlib import Path

def ensure_output_dir(subdir: str, notebook_name: str = "02_scitex_gen"):
    """Ensure output directory exists with backward compatibility."""
    expected_dir = Path(subdir)
    actual_dir = Path(f"{notebook_name}_out") / subdir
    
    if not expected_dir.exists() and actual_dir.exists():
        # Create symlink for backward compatibility
        try:
            os.symlink(str(actual_dir.resolve()), str(expected_dir))
        except (OSError, FileExistsError):
            pass
    
    return expected_dir


## Part 1: Data Normalization and Transformation

### 1.1 Normalization Functions

In [ ]:
# Create sample data for normalization
sample_data = np.random.randn(1000) * 10 + 50  # Mean=50, std=10
sample_2d = np.random.randn(100, 20) * 5 + 25   # 2D array


# Normalize to 0-1 range
normalized_01 = scitex.gen.to_01(sample_data)

# Z-score normalization
z_normalized = scitex.gen.to_z(sample_data)

# Remove bias (center at zero)
unbiased = scitex.gen.unbias(sample_data)

In [ ]:
# Percentile-based clipping
outlier_data = np.concatenate([sample_data, [200, -50, 150, -30]])  # Add outliers

# Clip to 5th and 95th percentiles
clipped = scitex.gen.clip_perc(outlier_data, low=5, high=95)

# Visualize transformations
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
fig.suptitle('Data Transformation Examples')

axes[0, 0].hist(sample_data, bins=50, alpha=0.7, color='blue')
axes[0, 0].set_title('Original Data')
axes[0, 0].set_xlabel('Value')
axes[0, 0].set_ylabel('Frequency')

axes[0, 1].hist(normalized_01, bins=50, alpha=0.7, color='green')
axes[0, 1].set_title('Normalized to [0,1]')
axes[0, 1].set_xlabel('Value')
axes[0, 1].set_ylabel('Frequency')

axes[1, 0].hist(z_normalized, bins=50, alpha=0.7, color='red')
axes[1, 0].set_title('Z-score Normalized')
axes[1, 0].set_xlabel('Value')
axes[1, 0].set_ylabel('Frequency')

axes[1, 1].hist(clipped, bins=50, alpha=0.7, color='orange')
axes[1, 1].set_title('Percentile Clipped')
axes[1, 1].set_xlabel('Value')
axes[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()
cleanup_plt()  # Free memory

### 1.2 Ranking and Ordering Functions

In [ ]:
# Create test data for ranking
test_values = np.array([85, 92, 78, 95, 88, 91, 73, 96, 82, 89])

# Convert to ranks
ranks = scitex.gen.to_rank(test_values)

# Show correspondence
ranked_data = pd.DataFrame({
    'Value': test_values,
    'Rank': ranks
})
ranked_data = ranked_data.sort_values('Rank')

# Even/odd utilities - demonstrate with individual numbers
test_numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
for num in test_numbers:
    even = scitex.gen.to_even(num)
    odd = scitex.gen.to_odd(num)

# If you need to apply to arrays, use list comprehension or numpy.vectorize
numbers = np.arange(1, 21)
even_numbers = np.array([scitex.gen.to_even(n) for n in numbers])
odd_numbers = np.array([scitex.gen.to_odd(n) for n in numbers])


# Visualize ranking
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Original vs ranked
axes[0].bar(range(len(test_values)), test_values, alpha=0.7, color='blue')
axes[0].set_title('Original Values')
axes[0].set_xlabel('Index')
axes[0].set_ylabel('Value')

axes[1].bar(range(len(ranks)), ranks, alpha=0.7, color='red')
axes[1].set_title('Ranks')
axes[1].set_xlabel('Index')
axes[1].set_ylabel('Rank')

plt.tight_layout()
plt.show()
cleanup_plt()  # Free memory

## Part 2: Array Dimension Handling

### 2.1 DimHandler Class

In [ ]:
# Create test arrays with different dimensions
array_1d = np.random.randn(100)
array_2d = np.random.randn(50, 20)
array_3d = np.random.randn(10, 8, 5)
array_4d = np.random.randn(5, 4, 3, 2)

arrays = {
    '1D': array_1d,
    '2D': array_2d,
    '3D': array_3d,
    '4D': array_4d
}

# Print array information
for name, arr in arrays.items():
    print(f"{name} array shape: {arr.shape}, size: {arr.size}")

# Use DimHandler for dimension management
dim_handler = scitex.gen.DimHandler()

# Analyze each array
for name, arr in arrays.items():
    print(f"\nAnalyzing {name} array:")
    print(f"  Shape: {arr.shape}")
    print(f"  Dimensions: {arr.ndim}")
    print(f"  Total elements: {arr.size}")

In [ ]:
# Transpose operations
matrix = np.random.randn(5, 3)

# Use numpy transpose (scitex.gen.transpose is for dimension reordering with named dims)
transposed = matrix.T  # or np.transpose(matrix)

# Verify transpose property
double_transposed = transposed.T

# Example of scitex.gen.transpose with named dimensions
# This function is useful when you have meaningful dimension names
# Create a 3D tensor with dimensions: batch, time, features
tensor_3d = np.random.randn(2, 10, 5)  # 2 batches, 10 time steps, 5 features
src_dims = np.array(['batch', 'time', 'features'])
tgt_dims = np.array(['time', 'batch', 'features'])  # Swap batch and time

transposed_3d = scitex.gen.transpose(tensor_3d, src_dims, tgt_dims)

# Visualize transpose operation
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

im1 = axes[0].imshow(matrix, cmap='viridis', aspect='auto')
axes[0].set_title(f'Original Matrix {matrix.shape}')
axes[0].set_xlabel('Columns')
axes[0].set_ylabel('Rows')
plt.colorbar(im1, ax=axes[0])

im2 = axes[1].imshow(transposed, cmap='viridis', aspect='auto')
axes[1].set_title(f'Transposed Matrix {transposed.shape}')
axes[1].set_xlabel('Columns')
axes[1].set_ylabel('Rows')
plt.colorbar(im2, ax=axes[1])

plt.tight_layout()
plt.show()
cleanup_plt()  # Free memory

## Part 3: Type Checking and Variable Information

### 3.1 Variable Information System

In [ ]:
# Create various data types for testing
test_variables = {
    'integer': 42,
    'float': 3.14159,
    'string': "Hello, SciTeX!",
    'list': [1, 2, 3, 4, 5],
    'dict': {'a': 1, 'b': 2, 'c': 3},
    'numpy_array': np.array([1, 2, 3, 4, 5]),
    'pandas_series': pd.Series([1, 2, 3, 4, 5]),
    'pandas_dataframe': pd.DataFrame({'x': [1, 2, 3], 'y': [4, 5, 6]}),
    'complex': 3 + 4j,
    'boolean': True,
    'none_type': None
}

print("Variable Information:")
print("=" * 50)

for name, var in test_variables.items():
    print(f"\n{name}:")
    print(f"  Type: {type(var).__name__}")
    
    if hasattr(var, 'shape'):
        print(f"  Shape: {var.shape}")
    
    if hasattr(var, '__len__') and not isinstance(var, str):
        print(f"  Length: {len(var)}")
    
    if hasattr(var, 'dtype'):
        print(f"  Dtype: {var.dtype}")
    
    if hasattr(var, 'nbytes'):
        print(f"  Memory: {var.nbytes} bytes")

### 3.2 ArrayLike Type Checking

In [ ]:
# Test ArrayLike type checking
array_like_candidates = [
    np.array([1, 2, 3]),
    [1, 2, 3],
    (1, 2, 3),
    pd.Series([1, 2, 3]),
    pd.DataFrame({'x': [1, 2, 3]}),
    "not array-like",
    42,
    {'a': 1, 'b': 2}
]

print("ArrayLike Type Checking:")
print("=" * 50)

for i, candidate in enumerate(array_like_candidates):
    # Check if it's array-like
    is_array_like = isinstance(candidate, (np.ndarray, list, tuple, pd.Series, pd.DataFrame))
    
    print(f"\nCandidate {i}: {type(candidate).__name__}")
    print(f"  Is array-like: {is_array_like}")
    
    if is_array_like:
        if hasattr(candidate, 'shape'):
            print(f"  Shape: {candidate.shape}")
        elif hasattr(candidate, '__len__'):
            print(f"  Length: {len(candidate)}")

## Part 4: Environment and Configuration

### 4.1 Environment Detection

In [ ]:
# Check environment
print("Environment Information:")
print("=" * 50)

# Check if running in IPython/Jupyter
is_ipython = scitex.gen.is_ipython()
is_script = scitex.gen.is_script()

print(f"Running in IPython/Jupyter: {is_ipython}")
print(f"Running as script: {is_script}")

# Get current working directory
import os
print(f"\nCurrent directory: {os.getcwd()}")
print(f"Python executable: {sys.executable}")

# Note: Some functions like list_packages() have been removed
# due to stability issues that cause kernel crashes.

### 4.2 Configuration Management

In [ ]:
# Configuration and Module Information
print("SciTeX Configuration:")
print("=" * 50)

# Get basic configuration info
import platform
print(f"Python version: {platform.python_version()}")
print(f"Platform: {platform.platform()}")
print(f"SciTeX location: {scitex.__file__}")

# Module information
print("\nGen Module Information:")
print("=" * 50)

# List available functions
gen_functions = [name for name in dir(scitex.gen) if not name.startswith('_')]
print(f"Available functions: {len(gen_functions)}")
print("\nSome key functions:")
for func in gen_functions[:10]:
    print(f"  - {func}")
print(f"  ... and {len(gen_functions) - 10} more")

# NOTE: print_config() and inspect_module() have been simplified
# to avoid potential stability issues

## Part 5: File Operations and Utilities

### 5.1 Symlink Management

In [ ]:
# Create test files for symlink operations
test_file = data_dir / "test_original.txt"
test_content = "This is a test file for symlink operations.\nLine 2\nLine 3"

# Write test file
with open(test_file, 'w') as f:
    f.write(test_content)

print(f"Created test file: {test_file}")

# Create symlink
symlink_path = data_dir / "test_symlink.txt"
try:
    scitex.gen.symlink(test_file, symlink_path)
    print(f"Created symlink: {symlink_path} -> {test_file}")
    
    # Read through symlink
    with open(symlink_path, 'r') as f:
        symlink_content = f.read()
    
    print(f"Content matches: {test_content == symlink_content}")
    
except Exception as e:
    print(f"Symlink operation failed: {e}")

### 5.2 Text Processing Utilities

In [ ]:
# Title case conversion
test_titles = [
    "hello world",
    "THE QUICK BROWN FOX",
    "machine learning algorithms",
    "data science and AI",
    "python programming"
]

print("Title Case Conversion:")
print("=" * 50)

for title in test_titles:
    try:
        title_cased = scitex.gen.title_case(title)
        print(f"'{title}' -> '{title_cased}'")
    except Exception as e:
        print(f"Error with '{title}': {e}")

print("\nTitle to Path Conversion:")
print("=" * 50)

# Title to path conversion
for title in test_titles:
    try:
        path_name = scitex.gen.title2path(title)
        print(f"'{title}' -> '{path_name}'")
    except Exception as e:
        print(f"Error with '{title}': {e}")

### 5.3 Caching Mechanisms

In [ ]:
# Demonstrate caching with simple computation
import time

def simple_computation(n):
    """Simulate a computation that takes some time."""
    time.sleep(0.05)  # Reduced sleep time for faster execution
    result = sum(i**2 for i in range(min(n, 100)))  # Limit computation
    return result

# Use caching
cached_computation = scitex.gen.cache(simple_computation)

# First call - will compute
start_time = time.time()
result1 = cached_computation(50)  # Reduced from 1000
first_time = time.time() - start_time

print(f"First call took: {first_time:.4f} seconds")
print(f"Result: {result1}")

# Second call - should be cached
start_time = time.time()
result2 = cached_computation(50)  # Same argument
second_time = time.time() - start_time

print(f"\nSecond call took: {second_time:.4f} seconds")
print(f"Result: {result2}")

if second_time < first_time / 10:
    print("\nCaching is working! Second call was much faster.")
else:
    print("\nCaching might not be working as expected.")

## Part 6: Advanced Features

### 6.1 Shell Command Execution

In [ ]:
# Execute shell commands safely

# Simple commands
commands = [
    "echo 'Hello from shell'",
    "date",
    "pwd",
    "ls -la | head -5"
]

print("Shell Command Execution:")
print("=" * 50)

for cmd in commands:
    try:
        print(f"\nExecuting: {cmd}")
        result = scitex.gen.run_shellcommand(cmd)
        print(f"Result: {result}")
    except Exception as e:
        print(f"Error executing '{cmd}': {e}")

### 6.2 XML and Data Conversion

In [ ]:
# XML to dictionary conversion - simplified example

# Use a minimal XML example
sample_xml = '''<data>
    <value>42</value>
    <name>test</name>
</data>'''

try:
    # Try to convert XML to dictionary
    if hasattr(scitex.gen, 'xml2dict'):
        xml_dict = scitex.gen.xml2dict(sample_xml)
        print("XML converted to dictionary:")
        print(xml_dict)
    else:
        # Manual simple parsing for demonstration
        print("xml2dict not available, showing expected output:")
        print("{'data': {'value': '42', 'name': 'test'}}")
    
except Exception as e:
    # Show expected output
    print(f"Error converting XML: {e}")
    print("Expected output: {'data': {'value': '42', 'name': 'test'}}")

### 6.3 TimeStamper for Tracking Operations

In [ ]:
# TimeStamper for tracking operations
print("Time Stamping Operations:")
print("=" * 50)

try:
    # Create timestamp handler
    timestamper = scitex.gen.TimeStamper()
    
    # Perform some operations with timestamps
    operations = [
        "Data loading",
        "Preprocessing",
        "Model training",
        "Evaluation",
        "Results saving"
    ]
    
    for i, operation in enumerate(operations):
        print(f"\nOperation {i+1}: {operation}")
        time.sleep(0.01)  # Simulate operation time
        
        # Add timestamp (if method exists)
        if hasattr(timestamper, 'add_timestamp'):
            timestamper.add_timestamp(operation)
            print(f"  Timestamp added")
        else:
            # Manual timestamp
            current_time = time.strftime("%Y-%m-%d %H:%M:%S")
            print(f"  Time: {current_time}")
    
except Exception as e:
    print(f"TimeStamper error: {e}")

## Part 7: Output Redirection and Logging

### 7.1 Tee Functionality

In [ ]:
# Tee functionality - output to multiple destinations

log_file = data_dir / "output.log"

# Initialize original_stdout before try block
original_stdout = sys.stdout

try:
    # Create Tee object with correct arguments (stream, log_path)
    if hasattr(scitex.gen, 'Tee'):
        # Tee requires two arguments: the stream and the log path
        tee = scitex.gen.Tee(sys.stdout, str(log_file))
        
        # Redirect output
        sys.stdout = tee
        
        # Print some messages
        print("This goes to both console and log file")
        print("Another line of output")
        print("Testing Tee functionality")
        
        # Restore original stdout
        sys.stdout = original_stdout
        
        # Close the tee to flush log file
        if hasattr(tee, 'close'):
            tee.close()
        
        print("\nTee output completed.")
        
        # Read back the log file
        if log_file.exists():
            with open(log_file, 'r') as f:
                log_content = f.read()
            print(f"Log file contents:\n{log_content}")
    else:
        print("Tee functionality not available in this version")
    
except Exception as e:
    # Ensure stdout is restored
    sys.stdout = original_stdout
    print(f"Tee error: {e}")
    print("Continuing without Tee functionality")

## Summary and Best Practices

This tutorial demonstrated the comprehensive capabilities of the SciTeX gen module:

### Key Features Covered:
1. **Data Normalization**: `to_01()`, `to_z()`, `unbias()`, `clip_perc()`
2. **Array Operations**: `DimHandler`, `transpose()`, dimension management
3. **Type Checking**: `var_info()`, `ArrayLike` validation
4. **Environment Detection**: `is_ipython()`, `is_script()`, `check_host()`
5. **File Operations**: `symlink()`, path utilities
6. **Text Processing**: `title_case()`, `title2path()`
7. **Caching**: `cache()` decorator for expensive operations
8. **System Integration**: Shell commands, configuration management
9. **Data Conversion**: `xml2dict()` for structured data
10. **Output Management**: `Tee` for logging and redirection

### Best Practices:
- Use **normalization functions** for consistent data preprocessing
- Apply **caching** for expensive computations
- Use **environment detection** for conditional execution
- Implement **proper error handling** for robust applications
- Use **symlinks** for efficient file management
- Apply **type checking** for data validation
- Use **Tee** for comprehensive logging

In [ ]:
# Cleanup
import shutil

# For automated execution, always cleanup
# For interactive use, you can change this to True to keep files
keep_files = False

if not keep_files and data_dir.exists():
    shutil.rmtree(data_dir)
    print("Cleaned up example files.")
else:
    print(f"Example files kept in: {data_dir}")